In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
df=pd.read_csv("/content/loan dataset.csv")
#loading the dataset

: 

Removing columns which are not logical and seems as outlier to the dataset


In [ ]:
df=df.drop(['Number of years of employment','Is there telephone number available','Number of years of stay in current address','Purpose of taking loan','Working abroad or not'],axis='columns')
df

: 

Changing the output columns from 1 and 2 to 0 and 1,so that it is very easy to handle the data.

In [ ]:
df['Loan Defaulted or not'] = df['Loan Defaulted or not'].apply(lambda x: x-1)
df

: 

By looking at the number of occurence of  specific values,reducing the number of categorical types of data to train the model well and to reduce overfitting.

In [ ]:
df['Owned property'].replace({'No property':'no','car or other property':'yes','Real Estate':'yes','building society savings agreement/life insurance':'yes'},inplace=True)
df['Owned property'].unique()


: 

In [ ]:
df['amount in savings account'].value_counts()
df['amount in savings account'].replace({'no savings account':'less than 100','between 500 and 1000':'above 500','greater than 1000':'above 500'},inplace=True)
df['amount in savings account'].unique()

: 

In [ ]:
df['Other loans plans taken'].replace({'stores':'yes','none':'no','bank':'yes'},inplace=True)

: 

In [ ]:
df['Type of job performed'].replace({'skilled employee / official':'skilled','unskilled - resident':'unskilled','management/ self-employed/highly qualified employee/ officer':'self employed','unemployed/ unskilled - non-resident':'unemployed'},inplace=True)
df['Type of job performed'].unique()

: 

In [ ]:
df['Type of Housing'].replace({'for free':'own'},inplace=True)
df['Type of Housing'].unique()

: 

In [ ]:
df['Marital Status'].replace({'female and divorced/seperated/married':'female','male and single':'male and single/divorced','male and divorced/seperated':'male and single/divorced','male and married/widowed':'male and married'},inplace=True)
df['Marital Status'].unique()

: 

In [ ]:
df['Loan History'].replace({'all loans at this bank paid back duly':'no loans taken/all loans paid back duly','critical account/other loans existing (not at this bank)':'critical account'},inplace=True)
df['Loan History'].unique()


: 

In [ ]:
df['amount in current account'].replace({'no current account':'less than 0/no account','less than 0':'less than 0/no account'},inplace=True)
df['amount in current account'].value_counts()

: 

In [ ]:
for column in df:
    if(df[column].dtype=='object'): 
        print(f'{column} : {df[column].unique()}')

: 

In [ ]:
df_dummies=pd.get_dummies(data=df,columns=['Loan History','Marital Status','amount in current account','amount in savings account','Other loans plans taken','Owned property','Type of job performed','Type of Housing','Guarantor or Debtor'])
df_dummies

: 

In [ ]:
cols_to_scale=['loan amount taken','Age of the applicant in Number of Years','time duration for loan']
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
df_dummies[cols_to_scale]=scaler.fit_transform(df_dummies[cols_to_scale])

: 

In [ ]:
x=df_dummies.drop('Loan Defaulted or not',axis=1)
y=df_dummies['Loan Defaulted or not']
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.1)
x_train

: 

In [ ]:
import tensorflow as tf
from tensorflow import keras
d=0.3
model1=keras.Sequential([
    keras.layers.Dense(60,input_shape=(32,),activation='relu'),
    keras.layers.Dense(500,activation='relu'),
    keras.layers.Dropout(d),
    keras.layers.Dense(100,activation='relu'),
    keras.layers.Dropout(d),
    keras.layers.Dense(10,activation='relu'),
    keras.layers.Dropout(d),
    
    keras.layers.Dense(1,activation='sigmoid')  
])
model1.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model1.fit(x_train,y_train,epochs=30,batch_size=8)
model1.evaluate(x_test,y_test)

: 

In [ ]:
y_pred = model1.predict(x_test).reshape(-1)
import numpy as np
y_pred=np.round(y_pred)
y_pred


: 

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

: 

In [ ]:
model1.fit(x_train,y_train,epochs=30,batch_size=8)
model1.evaluate(x_test,y_test)

: 

In [ ]:
from imblearn.over_sampling import SMOTE
smote=SMOTE(sampling_strategy='minority')
x_sm,y_sm=smote.fit_resample(x,y)
y_sm.value_counts()
from sklearn.model_selection import train_test_split
x_sm_train,x_sm_test,y_sm_train,y_sm_test=train_test_split(x_sm,y_sm,test_size=0.1,random_state=10)
model1.fit(x_sm_train,y_sm_train)
model1.evaluate(x_sm_test,y_sm_test)

: 

In [ ]:
yp_sm=model1.predict(x_sm_test)
yp_sm=np.round(yp_sm)

: 

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_sm_test,yp_sm))

: 

In [ ]:
cm=tf.math.confusion_matrix(labels=y_sm_test,predictions=yp_sm)
plt.figure(figsize=(10,7))
import seaborn as sns
sns.heatmap(cm,annot=True,fmt='d', cmap=plt.cm.CMRmap_r)
plt.xlabel('predicted')
plt.ylabel('truth')    

: 

In [ ]:
import pickle
with open('loan_dataset.pickle','wb') as f:
  pickle.dump(model1,f)


: 

In [ ]:
r=x_sm_test.sample(1)
y_sm_test[140]
r


: 

In [ ]:
model1.predict(r)

: 

In [ ]:
cm=tf.math.confusion_matrix(labels=y_test,predictions=y_pred)
plt.figure(figsize=(10,7))
import seaborn as sns
sns.heatmap(cm,annot=True,fmt='d', cmap=plt.cm.CMRmap_r)
plt.xlabel('predicted')
plt.ylabel('truth')

: 

In [ ]:
plt.figure(figsize=(12,10))
plt.scatter(yp,y_test)

: 